In [1]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 2а_ без сокращений.txt
    ├── 2б_Изъяты лексемы с частотой выше 100.txt
    ├── 2в_Изъяты лексемы с частотой выше 49.txt
    ├── 2г_Изъяты лексемы с частотой выше 29.txt
    └── 2д_Изъяты лексемы 

In [2]:
#├── Сокращение по частям речи ***
#│   ├── 1.Первый жанр исходная выборка.txt
#│   ├── 2.Первый жанр без клауз, включающих наречия.txt
#│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
#│   ├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt
#│   ├── Без прилагательных второй жанр.txt
#│   ├── Без прилагательных первый жанр.txt
#│   └── Случайные выборки.txt
#└── сокращение по частотности 
#    ├── 1а_ без сокращений.txt
#    ├── 1б_Изъяты лексемы с частотой выше 100.txt
#    ├── 1в_Изъяты лексемы с частотой выше 49.txt
#    ├── 1г_Изъяты лексемы с частотой выше 29.txt
#    ├── 1д_Изъяты лексемы с частотой выше 9.txt
#    ├── 2а_ без сокращений.txt
#    ├── 2б_Изъяты лексемы с частотой выше 100.txt
#    ├── 2в_Изъяты лексемы с частотой выше 49.txt
#    ├── 2г_Изъяты лексемы с частотой выше 29.txt
#    └── 2д_Изъяты лексемы с частотой выше 9.txt

#moditifcacion para PARTES
import re
import razdel
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from isanlp.pipeline_common import PipelineCommon
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm
import gc
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from razdel import sentenize
import nltk
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')


def cargar_datos(archivo, etiqueta):
    with open(archivo, 'r', encoding='utf-8') as f:
        textos = f.readlines()
    return [(texto.strip(), etiqueta) for texto in textos]

def tokenizar_fuction(texto):
    """Tokeniza el texto eliminando puntuación y convirtiendo a minúsculas."""
    texto_preprocesado = re.sub(r'[^\w\s]', '', texto.lower())
    tokens = nltk.word_tokenize(texto_preprocesado)
    return tokens

def calcular_similitud_mayoria_optimizado(textos1, textos2, threshold):
    """
        Calcula similitudes entre dos listas de textos usando GPU.
        Args:
            textos1: Lista de textos (original_datos_procesados).
            textos2: Lista de textos (train_data).
        Returns:
            Matriz booleana [len(textos1), len(textos2)] indicando similitudes.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")

    # Preprocesar todos los textos en batch
    textos1_preprocesados = [' '.join(tokenizar_fuction(t)) for t in textos1]
    textos2_preprocesados = [' '.join(tokenizar_fuction(t)) for t in textos2]

    # Crear representación binaria de tokens
    vectorizer = CountVectorizer(binary=True, tokenizer=nltk.word_tokenize)
    vectorizer.fit(textos1_preprocesados + textos2_preprocesados)

    # Convertir a matrices de tokens
    matriz_tokens1 = vectorizer.transform(textos1_preprocesados).toarray()
    matriz_tokens2 = vectorizer.transform(textos2_preprocesados).toarray()

    # Mover a GPU
    matriz_tokens1 = torch.from_numpy(matriz_tokens1).to(device)
    matriz_tokens2 = torch.from_numpy(matriz_tokens2).to(device)

     # Calcular intersecciones
    interseccion = torch.matmul(matriz_tokens1.float(), matriz_tokens2.T.float())

    # Tamaños de los conjuntos de tokens
    tamano_tokens1 = matriz_tokens1.sum(dim=1).unsqueeze(1)
    tamano_tokens2 = matriz_tokens2.sum(dim=1).unsqueeze(0)

    # Evitar división por cero
    tamano_tokens1 = tamano_tokens1.clamp(min=1)
    tamano_tokens2 = tamano_tokens2.clamp(min=1)

     # Calcular proporciones
    prop_1_en_2 = interseccion / tamano_tokens2
    prop_2_en_1 = interseccion / tamano_tokens1

    similitud = (prop_1_en_2 > threshold) | (prop_2_en_1 > threshold)





    return similitud

class DataLoader_raw:
    def __init__(self, path1, path2):
        self.path1 = path1
        self.path2 = path2
        self.path_original1 = '../dataset/Первый_жанр_исходная.txt'
        self.path_original2 = '../dataset/Второй_жанр_исходная.txt'    
    
    def __call__(self):
        datos_genero1 = cargar_datos(self.path1, 0)
        datos_genero2 = cargar_datos(self.path2, 1)
        datos = datos_genero1 + datos_genero2
        
        datos_original1 = cargar_datos(self.path_original1, 0)
        datos_original2 = cargar_datos(self.path_original2, 1)
        datos_originales = datos_original1 + datos_original2
        
        return {
            'datos_raw': datos,
            'original_datos_raw': datos_originales
        }

class SentenceSplitterAndCleaner:
    def __init__(self, tokenizer, min_length_threshold=6):
        """Inicializa el procesador con un tokenizador y un umbral de longitud mínima."""
        self.tokenizer = tokenizer
        self.min_length_threshold = min_length_threshold
    
    def _clean_and_split_text(self, texto):
        """
        Limpia y divide un texto en oraciones procesadas.
        
        Args:
            texto (str): Texto crudo a procesar.
        
        Returns:
            list: Lista de oraciones limpias.
        """
        # Paso 1: Marcar puntos para evitar fusiones no deseadas
        texto_limpio = re.sub(r'\.,', '. Ok999999999 ,', texto)  # Caso 1: Punto seguido de coma
        texto_limpio = re.sub(r'\.;', '. Ok999999999 ', texto_limpio)  # Caso 2: Punto seguido de punto y coma
        texto_limpio = re.sub(r'\. ([a-zа-я])', r'. Ok999999999 \1', texto_limpio)  # Caso 3: Punto seguido de minúscula
        texto_limpio = re.sub(r'(\w)([А-Я])', r'\1. \2', texto_limpio)  # Caso 4: Minúscula seguida de mayúscula
        
        #creo que hay que agregar una nueva condivion  дл.,0 
        #", 2-3 см шир.",0 
        #", 1 см шир.",0 si hay un punto uego numeros quitar el punto
        
        # Paso 2: Eliminar corchetes y dividir en oraciones
        texto_sin_corchetes = re.sub(r'\[.*?\]', '', texto_limpio).strip()
        oraciones = [oracion.text for oracion in sentenize(texto_sin_corchetes)]
        
        # Paso 3: Restaurar espacios y limpiar marcadores temporales
        oraciones_limpias = [re.sub(r'\s*Ok999999999', ' ', oracion).strip() for oracion in oraciones]
        
        return oraciones_limpias
    
    def _process_sentences(self, datos, target_list):
        """
        Procesa un conjunto de datos y agrega oraciones válidas a la lista objetivo.
        
        Args:
            datos (list): Lista de tuplas (texto, etiqueta).
            target_list (list): Lista donde se almacenarán las oraciones procesadas.
        """
        for texto, etiqueta in datos:
            oraciones = self._clean_and_split_text(texto)
            for oracion in oraciones:
                # Filtrar oraciones cortas basadas en el número de tokens
                if len(self.tokenizer.encode(oracion, truncation=False)) >= self.min_length_threshold:
                    target_list.append((oracion, etiqueta))
    
    def __call__(self, data_raw, original_data_raw):
        """
        Procesa los datos crudos y originales, devolviendo dos conjuntos limpios.
        
        Args:
            data_raw (list): Datos modificados crudos.
            original_data_raw (list): Datos originales crudos.
        
        Returns:
            dict: Diccionario con datos procesados y originales procesados.
        """
        datos_procesados = []
        original_datos_procesados = []
        
        # Procesar datos modificados
        self._process_sentences(data_raw, datos_procesados)
        
        # Procesar datos originales
        self._process_sentences(original_data_raw, original_datos_procesados)
        
        return {
            'datos_procesados': datos_procesados,
            'original_datos_procesados': original_datos_procesados
        }
    
class DataProcessor:
    #def __init__(self, tokenizer, max_length, random_state, test_size=0.2):
    def __init__(self,
                 tokenizer,
                 max_length,
                 random_state,
                 test_size=0.2,
                 name='сокращение по частотности',
                 threshold = 0.5
                ):
    
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.test_size = test_size
        self.random_state = random_state
        self.name = name
        self.threshold = threshold
    
    def __call__(self, datos_procesados, original_datos_procesados):
        #datos_procesados = data['datos_procesados']
        df = pd.DataFrame(datos_procesados, columns=["text", "label"])


        train_data, test_data = train_test_split(df, test_size=self.test_size, random_state=self.random_state)
        #generar un nuevo test data
        print("test_data:", len(test_data))
        train_texts = train_data["text"].tolist()
        original_texts = [oracion for oracion, _ in original_datos_procesados]
        original_labels = [etiqueta for _, etiqueta in original_datos_procesados]
        
        similitudes = calcular_similitud_mayoria_optimizado(original_texts,
                                                            train_texts,
                                                            self.threshold)  # [len(original), len(train)]
        
        interseccion = similitudes.any(dim=1)  # True si hay intersección con algún train_text
        mask = ~interseccion.cpu().numpy()
        nuevo_test = [(texto, etiqueta) for texto, etiqueta, keep in zip(original_texts, original_labels, mask) if keep]
        print("Tamaño de nuevo_test:", len(nuevo_test))
         
        if self.random_state == 0:
            train_data.to_csv(f'Сокращение по частям речи/train_{self.name}', index=False)
            print(f"train_{self.name}.csv")
            nuevo_test_df = pd.DataFrame(nuevo_test, columns=["text", "label"])
            
            nuevo_test_df.to_csv(f'Сокращение по частям речи/test_{self.name}', index=False)
            print(f"test_{self.name}.csv")
        
        #no se a niandido el nuevo test
        # Tokenizar entrenamiento
        train_encodings = self.tokenizer(
            train_data["text"].tolist(),
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        train_labels = torch.tensor(train_data["label"].values)
        # Tokenizar prueba
        test_encodings = self.tokenizer(
            [texto for texto, _ in nuevo_test],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        test_labels = torch.tensor([etiqueta for _, etiqueta in nuevo_test])
        return {
            'train_encodings': train_encodings,
            'train_labels': train_labels,
            'test_encodings': test_encodings,
            'test_labels': test_labels
        }  
    
    
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

class DatasetCreator:
    def __init__(self, batch_size=8):
        self.batch_size = batch_size
    
    def __call__(self, train_encodings, train_labels, test_encodings, test_labels):
        train_dataset = TextDataset(train_encodings, train_labels)
        test_dataset = TextDataset(test_encodings, test_labels)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)
        return {'train_loader': train_loader, 'test_loader': test_loader}
    
def train_model(model, train_loader, optimizer, loss_fn, device, epochs=3):
    for epoch in range(epochs):
        model.train()
        loop = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}')
        for batch in loop:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            loop.set_postfix(loss=loss.item())
        torch.cuda.empty_cache()#libera memroria de la gpu
        gc.collect()#limpa la cpu

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = sum(p == l for p, l in zip(all_preds, all_labels)) / len(all_labels)
    report = classification_report(all_labels, all_preds, target_names=["Género 1", "Género 2"])
    conf_matrix = confusion_matrix(all_labels, all_preds)
    return {"accuracy": accuracy, "report": report, "conf_matrix": conf_matrix}

def display_results(results):
    print(f'Precisión en el conjunto de prueba: {results["accuracy"]:.2%}')
    print("\nReporte de clasificación:\n", results["report"])
    plt.figure(figsize=(6, 4))
    sns.heatmap(results["conf_matrix"], annot=True, fmt='d', cmap='Blues', 
                xticklabels=["Género 1", "Género 2"], 
                yticklabels=["Género 1", "Género 2"])
    plt.title('Matriz de Confusión')
    plt.xlabel('Predicción')
    plt.ylabel('Real')
    plt.show()

/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-24 12:55:18.268615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#├── Сокращение по частям речи 
#│   ├── 1.Первый жанр исходная выборка.txt
#│   ├── 2.Первый жанр без клауз, включающих наречия.txt
#│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
#│   ├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt
#│   ├── Без прилагательных второй жанр.txt ***
#│   ├── Без прилагательных первый жанр.txt ***
#│   └── Случайные выборки.txt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
max_length = 128
#Longitud promedio de tokens: 27.77
#Longitud máxima de tokens: 192
#Longitud mínima de tokens: 2
#falta ver la logitud real del nuevo dataset
batch_size = 8
epochs = 3
num_repeats = 6
seeds = list(range(num_repeats))


accuracies = []
for seed in seeds:
    print(f"Repetición con semilla {seed}")
    
    # Instanciar DataProcessor con la semilla actual
    #data_processor = DataProcessor(tokenizer, max_length, test_size=0.2, random_state=seed)
    
    # Crear el pipeline (ajusta según tu estructura)
    ppl = PipelineCommon([
        (DataLoader_raw('../dataset/Сокращение по частям речи/Без прилагательных первый жанр.txt',
                        '../dataset/Сокращение по частям речи/Без прилагательных второй жанр.txt'), 
         [], 
         {'datos_raw': 'datos_raw',
         'original_datos_raw': 'original_datos_raw'}),
        (SentenceSplitterAndCleaner(tokenizer), 
         ['datos_raw','original_datos_raw'], 
         {'datos_procesados': 'datos_procesados',
         'original_datos_procesados': 'original_datos_procesados'}),
        (DataProcessor(tokenizer,
                       max_length,
                       seed,
                       name = 'Без прилагательных',
                       threshold = 0.5),  #217 - 195 -> aceptable
        
         ['datos_procesados','original_datos_procesados'], 
         {'train_encodings': 'train_encodings', 'train_labels': 'train_labels', 
          'test_encodings': 'test_encodings', 'test_labels': 'test_labels'}),
        (DatasetCreator(batch_size=batch_size), 
         ['train_encodings', 'train_labels', 'test_encodings', 'test_labels'], 
         {'train_loader': 'train_loader', 'test_loader': 'test_loader'})
    ])
    
    # Ejecutar el pipeline
    result = ppl()
    train_loader = result['train_loader']
    test_loader = result['test_loader']
    
    # Configurar el modelo (reiniciarlo en cada repetición)
    model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=2)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    # Entrenar el modelo (función train_model definida por ti)
    train_model(model, train_loader, optimizer, loss_fn, device, epochs=epochs)
    
    # Evaluar el modelo (función evaluate_model definida por ti)
    results = evaluate_model(model, test_loader, device)
    accuracy = results['accuracy']  # Ajusta según la métrica que uses
    accuracies.append(accuracy)
    
    print(f"Accuracy in {seed} seed: {accuracy:.4f}")

# Calcular promedio y desviación estándar
average_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print(f"\nAccuracy promedio después de {num_repeats} repeticiones: {average_accuracy:.4f} ± {std_accuracy:.4f}")

/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Repetición con semilla 0
test_data: 207
Usando dispositivo: cuda


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Tamaño de nuevo_test: 210
train_Без прилагательных.csv
test_Без прилагательных.csv


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

...

Accuracy in 4 seed: 0.9843
Repetición con semilla 5
test_data: 207
Usando dispositivo: cuda


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Tamaño de nuevo_test: 196


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

Accuracy in 5 seed: 1.0000

Accuracy promedio después de 6 repeticiones: 0.9932 ± 0.0070


In [4]:
#├── Сокращение по частям речи 
#│   ├── 1.Первый жанр исходная выборка.txt
#│   ├── 2.Первый жанр без клауз, включающих наречия.txt ***
#│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
#│   ├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt
#│   ├── Без прилагательных второй жанр.txt 
#│   ├── Без прилагательных первый жанр.txt 
#│   └── Случайные выборки.txt

accuracies = []
for seed in seeds:
    print(f"Repetición con semilla {seed}")
    
    # Instanciar DataProcessor con la semilla actual
    #data_processor = DataProcessor(tokenizer, max_length, test_size=0.2, random_state=seed)
    
    # Crear el pipeline (ajusta según tu estructura)
    ppl = PipelineCommon([
        (DataLoader_raw('../dataset/Сокращение по частям речи/2.Первый жанр без клауз, включающих наречия.txt',
                        '../dataset/Второй_жанр_исходная.txt'), 
         [], 
         {'datos_raw': 'datos_raw',
         'original_datos_raw': 'original_datos_raw'}),
        (SentenceSplitterAndCleaner(tokenizer), 
         ['datos_raw','original_datos_raw'], 
         {'datos_procesados': 'datos_procesados',
         'original_datos_procesados': 'original_datos_procesados'}),
        (DataProcessor(tokenizer,
                       max_length,
                       seed,
                       name = '2.Первый жанр без клауз, включающих наречия',
                       threshold = 0.5),  #217 - 195 -> aceptable
        
         ['datos_procesados','original_datos_procesados'], 
         {'train_encodings': 'train_encodings', 'train_labels': 'train_labels', 
          'test_encodings': 'test_encodings', 'test_labels': 'test_labels'}),
        (DatasetCreator(batch_size=batch_size), 
         ['train_encodings', 'train_labels', 'test_encodings', 'test_labels'], 
         {'train_loader': 'train_loader', 'test_loader': 'test_loader'})
    ])
    
    # Ejecutar el pipeline
    result = ppl()
    train_loader = result['train_loader']
    test_loader = result['test_loader']
    
    # Configurar el modelo (reiniciarlo en cada repetición)
    model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=2)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    # Entrenar el modelo (función train_model definida por ti)
    train_model(model, train_loader, optimizer, loss_fn, device, epochs=epochs)
    
    # Evaluar el modelo (función evaluate_model definida por ti)
    results = evaluate_model(model, test_loader, device)
    accuracy = results['accuracy']  # Ajusta según la métrica que uses
    accuracies.append(accuracy)
    
    print(f"Accuracy in {seed} seed: {accuracy:.4f}")

# Calcular promedio y desviación estándar
average_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print(f"\nAccuracy promedio después de {num_repeats} repeticiones: {average_accuracy:.4f} ± {std_accuracy:.4f}")

Repetición con semilla 0
test_data: 220
Usando dispositivo: cuda


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Tamaño de nuevo_test: 174
train_2.Первый жанр без клауз, включающих наречия.csv
test_2.Первый жанр без клауз, включающих наречия.csv


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

Accuracy in 0 seed: 0.9943
Repetición con semilla 1
test_data: 220
Usando dispositivo: cuda


...

Accuracy in 4 seed: 0.9886
Repetición con semilla 5
test_data: 220
Usando dispositivo: cuda


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Tamaño de nuevo_test: 165


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

Accuracy in 5 seed: 0.9818

Accuracy promedio después de 6 repeticiones: 0.9902 ± 0.0045


In [5]:
#├── Сокращение по частям речи 
#│   ├── 1.Первый жанр исходная выборка.txt
#│   ├── 2.Первый жанр без клауз, включающих наречия.txt 
#│   ├── 3.Первый жанр без клауз, включающих глаголы.txt ***
#│   ├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt
#│   ├── Без прилагательных второй жанр.txt 
#│   ├── Без прилагательных первый жанр.txt 
#│   └── Случайные выборки.txt


accuracies = []
for seed in seeds:
    print(f"Repetición con semilla {seed}")
    
    # Instanciar DataProcessor con la semilla actual
    #data_processor = DataProcessor(tokenizer, max_length, test_size=0.2, random_state=seed)
    
    # Crear el pipeline (ajusta según tu estructura)
    ppl = PipelineCommon([
        (DataLoader_raw('../dataset/Сокращение по частям речи/3.Первый жанр без клауз, включающих глаголы.txt',
                        '../dataset/Второй_жанр_исходная.txt'), 
         [], 
         {'datos_raw': 'datos_raw',
         'original_datos_raw': 'original_datos_raw'}),
        (SentenceSplitterAndCleaner(tokenizer), 
         ['datos_raw','original_datos_raw'], 
         {'datos_procesados': 'datos_procesados',
         'original_datos_procesados': 'original_datos_procesados'}),
        (DataProcessor(tokenizer,
                       max_length,
                       seed,
                       name = '3.Первый жанр без клауз, включающих глаголы',
                       threshold = 0.5),  #217 - 195 -> aceptable
        
         ['datos_procesados','original_datos_procesados'], 
         {'train_encodings': 'train_encodings', 'train_labels': 'train_labels', 
          'test_encodings': 'test_encodings', 'test_labels': 'test_labels'}),
        (DatasetCreator(batch_size=batch_size), 
         ['train_encodings', 'train_labels', 'test_encodings', 'test_labels'], 
         {'train_loader': 'train_loader', 'test_loader': 'test_loader'})
    ])
    
    # Ejecutar el pipeline
    result = ppl()
    train_loader = result['train_loader']
    test_loader = result['test_loader']
    
    # Configurar el modelo (reiniciarlo en cada repetición)
    model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=2)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    # Entrenar el modelo (función train_model definida por ti)
    train_model(model, train_loader, optimizer, loss_fn, device, epochs=epochs)
    
    # Evaluar el modelo (función evaluate_model definida por ti)
    results = evaluate_model(model, test_loader, device)
    accuracy = results['accuracy']  # Ajusta según la métrica que uses
    accuracies.append(accuracy)
    
    print(f"Accuracy in {seed} seed: {accuracy:.4f}")

# Calcular promedio y desviación estándar
average_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print(f"\nAccuracy promedio después de {num_repeats} repeticiones: {average_accuracy:.4f} ± {std_accuracy:.4f}")


Repetición con semilla 0
test_data: 205
Usando dispositivo: cuda


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Tamaño de nuevo_test: 222
train_3.Первый жанр без клауз, включающих глаголы.csv
test_3.Первый жанр без клауз, включающих глаголы.csv


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

Accuracy in 0 seed: 0.9820
Repetición con semilla 1
test_data: 205
Usando dispositivo: cuda


...

Accuracy in 4 seed: 0.9710
Repetición con semilla 5
test_data: 205
Usando dispositivo: cuda


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Tamaño de nuevo_test: 222


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

Accuracy in 5 seed: 0.9775

Accuracy promedio después de 6 repeticiones: 0.9790 ± 0.0105


In [6]:
#├── Сокращение по частям речи 
#│   ├── 1.Первый жанр исходная выборка.txt
#│   ├── 2.Первый жанр без клауз, включающих наречия.txt 
#│   ├── 3.Первый жанр без клауз, включающих глаголы.txt 
#│   ├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt ***
#│   ├── Без прилагательных второй жанр.txt 
#│   ├── Без прилагательных первый жанр.txt 
#│   └── Случайные выборки.txt


accuracies = []
for seed in seeds:
    print(f"Repetición con semilla {seed}")
    
    # Instanciar DataProcessor con la semilla actual
    #data_processor = DataProcessor(tokenizer, max_length, test_size=0.2, random_state=seed)
    
    # Crear el pipeline (ajusta según tu estructura)
    ppl = PipelineCommon([
        (DataLoader_raw('../dataset/Сокращение по частям речи/4. Первый жанр без клауз, включающих глаголы и наречия.txt',
                        '../dataset/Второй_жанр_исходная.txt'), 
         [], 
         {'datos_raw': 'datos_raw',
         'original_datos_raw': 'original_datos_raw'}),
        (SentenceSplitterAndCleaner(tokenizer), 
         ['datos_raw','original_datos_raw'], 
         {'datos_procesados': 'datos_procesados',
         'original_datos_procesados': 'original_datos_procesados'}),
        (DataProcessor(tokenizer,
                       max_length,
                       seed,
                       name = '4.Первый жанр без клауз, включающих глаголы и наречия',
                       threshold = 0.5),  #217 - 195 -> aceptable
        
         ['datos_procesados','original_datos_procesados'], 
         {'train_encodings': 'train_encodings', 'train_labels': 'train_labels', 
          'test_encodings': 'test_encodings', 'test_labels': 'test_labels'}),
        (DatasetCreator(batch_size=batch_size), 
         ['train_encodings', 'train_labels', 'test_encodings', 'test_labels'], 
         {'train_loader': 'train_loader', 'test_loader': 'test_loader'})
    ])
    
    # Ejecutar el pipeline
    result = ppl()
    train_loader = result['train_loader']
    test_loader = result['test_loader']
    
    # Configurar el modelo (reiniciarlo en cada repetición)
    model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=2)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    # Entrenar el modelo (función train_model definida por ti)
    train_model(model, train_loader, optimizer, loss_fn, device, epochs=epochs)
    
    # Evaluar el modelo (función evaluate_model definida por ti)
    results = evaluate_model(model, test_loader, device)
    accuracy = results['accuracy']  # Ajusta según la métrica que uses
    accuracies.append(accuracy)
    
    print(f"Accuracy in {seed} seed: {accuracy:.4f}")

# Calcular promedio y desviación estándar
average_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print(f"\nAccuracy promedio después de {num_repeats} repeticiones: {average_accuracy:.4f} ± {std_accuracy:.4f}")


Repetición con semilla 0
test_data: 200
Usando dispositivo: cuda


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Tamaño de nuevo_test: 237
train_4.Первый жанр без клауз, включающих глаголы и наречия.csv
test_4.Первый жанр без клауз, включающих глаголы и наречия.csv


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

Accuracy in 0 seed: 0.9789
Repetición con semilla 1
test_data: 200
Usando dispositivo: cuda


...

Accuracy in 4 seed: 0.9828
Repetición con semilla 5
test_data: 200
Usando dispositivo: cuda


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Tamaño de nuevo_test: 231


/workspace/notebooks/Carlos/fine_tunig_project/mi_entorno/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

Accuracy in 5 seed: 0.9697

Accuracy promedio después de 6 repeticiones: 0.9771 ± 0.0055
